In [18]:
!pip3 install torch torchvision torchaudio
!pip3 install transformers
from sentence_transformers import SentenceTransformer
from sentence_transformers import util
from transformers import pipeline
from tqdm import tqdm
import torch
import json
import requests
import textwrap

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You should consider upgrading via the '/Users/arush/workspace/646proj/LaMP/data/flant5/bin/python3 -m pip install --upgrade pip' command.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You should consider upgrading via the '/Users/arush/workspace/646proj/LaMP/data/flant5/bin/python3 -m pip install --upgrade pip' command.


In [34]:
with open("/Users/arush/workspace/646proj/LaMP/data/sampled_rating_20.json", "r") as f:
    json_data = json.load(f)

In [10]:
model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')

In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
llm_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
model_id = "google/flan-t5-xl"
summarizer = pipeline("summarization", model = model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [35]:
i = 0
golds = []

for user in tqdm(json_data):
    i += 1
    corpus = []
    scores = []
    profile = user['profile']
    query = user['input']

    input_prompt = "Your task is to answer the query based on user's relevant history interactions.\nHere’s the query: {}\n".format(query)
    past_interactions = []
    ratings_with_reviews = {str(i): [] for i in range(1, 6)} 

    for doc in profile:
        corpus.append(doc['text'])
        scores.append(doc['score'])

    corpus_embeddings = model.encode(corpus, convert_to_tensor=True)
    query_embedding = model.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5, score_function=util.dot_score)
    hits = hits[0]
    predictions = {}

    for hit in hits:
        score = str(scores[hit['corpus_id']])  # Convert to string
        ratings_with_reviews[score].append(corpus[hit['corpus_id']])

    word_limit = 512
    for key, value in ratings_with_reviews.items():
        all_reviews = ""
        if value:
            sentences_str = " ".join(value)
            words_so_far = len(all_reviews.split())
            words_in_current = len(sentences_str.split())
    
            if words_so_far + words_in_current <= word_limit:
                all_reviews += sentences_str + " "
            else:
                break

            final_result = " ".join(all_reviews.split()[:word_limit])
            input_prompt += "Product rating given by the user for the following review: '{}' is '{}'".format(summarizer(final_result)[0]['summary_text'], key) + "\n"
        else:
            break
            
    inputs = tokenizer(input_prompt, return_tensors="pt")
    outputs = llm_model.generate(**inputs)
    predictions["output"] = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    predictions["id"] = user["id"]
    golds.append(predictions)

final_json = {}
final_json["task"] = "LaMP_3"
final_json["golds"] = golds

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [19:57<00:00, 59.85s/it]


In [36]:
with open("pred_summarized_profile_product_rating.json", "w") as f:
    json.dump(final_json, f)